MLP FOR A BINARY CLASSIFICATION PROBLEM: Physical or Non physical matrix?

The goal is to descriminate between matrices that are physical, i.e. that represent a quantum state, or matrices that are not physical. In this first approach I use matrix with all the same size, associated to an Hilbert Space Dimension of 4: 4x4 matrices, i.e. 16 complex elements in each matrix. 
I generated 300 physical and 300 non physical matrices in a MATLAB code (Dataset_creation.m). A physical matrix, also called Density matrix, must satisfy three requirements: it must be positive definite, hermitian, with trace equal to 1. In order to obtain this, in the MATLAB code, I first randomly generate 4x4 complex matrices. Each element of these matrices has real and imaginary part randomly extracted from a uniform distribution in the interval (-1,1). Then, I make matrices symmetric, with real diagonal values. To obtain positive definite matrices I use the Modified Cholesky Factorization. I found on internet a MATLAB code to implement this factorization (modchol_ldlt.m), for this I chose to generate matrices in MATLAB. Finally, I make matrices hermitian, and I normalize them to obtain trace equal to 1.
I also created non physical matrices. From these we can found: 75 randomly generated complex matrices, 75 positive definite complex matrices, 75 positive definite and hermitian matrices and 75 positive definite matrices with trace equal to 1. 
I reshape each generated matrix in a row vector of shape (1x16). From the MATLAB code I saved real and imaginary part of Physical and Non Physical matrices in four '.txt' files. Each row of files represents a different matrix, the columns are matrices' elements (my features).
I will build the complete dataset in this code. 

I choose label 0 to classify Physical matrices, label 1 to classify Non Physical matrices.

I will implement a Multilayer Perceptron to solve the classification problem. 

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import cmath

DATASET CREATION

To create the dataset, first I import the generated data in MATLAB. I recombine the corresponding real and imaginary part of each element of each matrix by store them in two matrix: Physical_mat and NonPhysical_mat. Each row of Physical_mat (NonPhysical_mat) represents a specific physical (non physical) matrix, while columns are the features (complex elements of matrices). Physical_mat and NonPhysical_mat have dimensions (300,16).
Then, I concatenate target vectors to the two matrices and I create the final dataset. The Dataset matrix has dimension (600,17) where 600 is the number of generate matrices (300 physical + 300 non physical), the first 16 columns have features while the last column has target labels (0 or 1). 

In [3]:
#import data generated in MATLAB
physical_real=np.loadtxt(r'/Users/noemitagliavacche/Documents/PhD/COURSES/MACHINE_LEARNING/Exam/MATLAB_dataset_creation/300_Physical_matrices_Dim=4_RealPart.txt')
Non_physical_real=np.loadtxt(r'/Users/noemitagliavacche/Documents/PhD/COURSES/MACHINE_LEARNING/Exam/MATLAB_dataset_creation/300_Non_Physical_matrices_Dim=4_RealPart.txt')
physical_imag=np.loadtxt(r'/Users/noemitagliavacche/Documents/PhD/COURSES/MACHINE_LEARNING/Exam/MATLAB_dataset_creation/300_Physical_matrices_Dim=4_ImagPart.txt')
Non_physical_imag=np.loadtxt(r'/Users/noemitagliavacche/Documents/PhD/COURSES/MACHINE_LEARNING/Exam/MATLAB_dataset_creation/300_Non_Physical_matrices_Dim=4_ImagPart.txt')
#print(physical_real.shape, Non_physical_real.shape)

################ TO CREATE THE DATASET #############
Physical_mat=[]
NonPhysical_mat=[]

for i in range(physical_real.shape[0]):
    for j in range(physical_real.shape[1]):
        Physical_mat=np.append(Physical_mat,complex(physical_real[i][j],physical_imag[i][j]))
        NonPhysical_mat=np.append(NonPhysical_mat,complex(Non_physical_real[i][j],Non_physical_imag[i][j]))
        
Physical_mat=np.reshape(Physical_mat, (physical_real.shape[0],physical_real.shape[1]))    
NonPhysical_mat=np.reshape(NonPhysical_mat, (physical_real.shape[0],physical_real.shape[1]))
#print(Physical_mat.shape, NonPhysical_mat.shape)

#Classifier: 0-> Physical matrix, 1-> Non physical matrix
target_Physical=np.zeros((Physical_mat.shape[0],1),dtype=int)
target_NonPhysical=np.ones((Physical_mat.shape[0],1),dtype=int)


Physical_data=np.concatenate((Physical_mat,target_Physical),axis=1)
NonPhysical_data=np.concatenate((NonPhysical_mat,target_NonPhysical),axis=1)

dataset=np.concatenate((Physical_data,NonPhysical_data),axis=0)
print("Dataset dimension : ", dataset.shape)
np.random.shuffle(dataset)

features=dataset[:,:-1]
target=[]
for elements in dataset[:,dataset.shape[1]-1]:
    target=np.append(target, int(elements.real))

target=target.T
print("features size: ", features.shape, ", target size: ", target.shape)




Dataset dimension :  (600, 17)
features size:  (600, 16) , target size:  (600,)


Now, I can split the dataset in train and validation set, with a ratio of 2:1

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(features, target, test_size=0.33, shuffle=False)
print("X_train: ",X_train.shape,", X_val: " ,X_val.shape, ", Y_train: ", Y_train.shape, ", Y_val: ",Y_val.shape)


X_train:  (402, 16) , X_val:  (198, 16) , Y_train:  (402,) , Y_val:  (198,)


BUILDING THE MLP MODEL

In [5]:
# keras imports
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Dropout, Flatten, Activation
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend as K
from tensorflow.keras import metrics
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TerminateOnNaN

: 

: 